In [1]:
import pandas as pd
import numpy as np
import datetime
import math

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8, progress_bar=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
# Set file paths
file_path = '/Volumes/EnergyData/AER/'

bid_dayoffer_path = f'{file_path}Raw_data_zipped/BIDDAYOFFER_D'
bid_peroffer_path = f'{file_path}Raw_data_zipped/BIDPEROFFER_D'
dispatch_load_path = f'{file_path}Raw_data_zipped/DISPATCHLOAD'
dispatch_regionsum_path = f'{file_path}Raw_data_zipped/DISPATCHREGIONSUM'
dispatch_price_path = f'{file_path}Raw_data_zipped/DISPATCHPRICE'

In [3]:
market_price_cap_mapping = dict()
market_price_cap_mapping['FY2017_2018'] = 14500
market_price_cap_mapping['FY2018_2019'] = 14700
market_price_cap_mapping['FY2019_2020'] = 15000
market_price_cap_mapping['FY2020_2021'] = 15100
market_price_cap_mapping['FY2021_2022'] = 15500
market_price_cap_mapping['FY2022_2023'] = 16600

In [4]:
# Get YearMonth to FY mapping dict
YearMonthFY = pd.read_excel(f'{file_path}Reference_data/duidinfo_SRMC.xlsx',
                         sheet_name='YearMonthFY').set_index(['Year','Month'])['FY'].to_dict()

# Define functions

In [5]:
def get_regional_duid_mapping(file_path,regionid):
    # Read the duid mapping table
    # It is obtained from Brian (from AER database)
    # Only SA1 DUIDs, covers both GENERATOR and LOAD
    duid_mapping = pd.read_csv(f'{file_path}Reference_data/duidinfo.csv',
                               usecols=['DUID','REGIONID','STATIONNAME','PARTICIPANTNAME',
                                        'DISPATCHTYPE','SCHEDULE_TYPE','FUELTYPE'])
    duid_mapping = duid_mapping[(duid_mapping['REGIONID']==regionid)&
                                (duid_mapping['SCHEDULE_TYPE'].isin(['SCHEDULED','SEMI-SCHEDULED']))]
    return duid_mapping

In [6]:
def get_generator_srmc(year,month,file_path,regionid):
    
    # Get YearMonth to FY mapping dict
    YearMonthFY = pd.read_excel(f'{file_path}Reference_data/duidinfo_SRMC.xlsx',
                             sheet_name='YearMonthFY').set_index(['Year','Month'])['FY'].to_dict()
    # Get the FY
    FY_use = YearMonthFY[(year,month)][2:]
    #     print(FY_use)
    
    # Get DUID SRMC (Only covers GENERATOR)
    SRMC_regional = pd.read_excel('/Volumes/EnergyData/AER/Reference_data/duidinfo_SRMC.xlsx',
                             sheet_name='SRMC',
                             usecols=['DUID','REGIONID','FUELTYPE',f'SRMC_{FY_use}'])\
                 .rename(columns={f'SRMC_{FY_use}':'SRMC'})
    SRMC_regional = SRMC_regional[SRMC_regional['REGIONID']==regionid]
    
    return SRMC_regional

In [7]:
# From DISPATCHLOAD get AVAILABILITY
def read_dispatchload(year,month,file_path,duid_mapping):

    temp_dispatch = pd.read_csv(f'{file_path}Raw_data_zipped/DISPATCHLOAD/PUBLIC_DVD_DISPATCHLOAD_{str(year)}{str(month).zfill(2)}010000.zip',
                                skiprows=1,
                                parse_dates=['SETTLEMENTDATE'],
                                usecols=['SETTLEMENTDATE','DUID','INTERVENTION',
                                         'TOTALCLEARED','AVAILABILITY','INITIALMW'])
    
    temp_dispatch = temp_dispatch[(temp_dispatch['DUID'].isin(duid_mapping['DUID'].values))&
                                  (temp_dispatch['INTERVENTION']==0)].drop(columns=['INTERVENTION'])
    
    return temp_dispatch.drop_duplicates()

In [8]:
def read_dispatchregionsum(year,month,file_path,regionid):
    temp_dispatchregionsum = pd.read_csv(f'{file_path}Raw_data_zipped/DISPATCHREGIONSUM/PUBLIC_DVD_DISPATCHREGIONSUM_{str(year)}{str(month).zfill(2)}010000.zip',
                                skiprows=1,
                                parse_dates=['SETTLEMENTDATE'],
                                usecols=['SETTLEMENTDATE','REGIONID','TOTALDEMAND','NETINTERCHANGE','DISPATCHABLELOAD',
                                         'INTERVENTION']).dropna(subset=['REGIONID'])
    temp_dispatchregionsum = temp_dispatchregionsum[(temp_dispatchregionsum['REGIONID']==regionid)&
                                                    (temp_dispatchregionsum['INTERVENTION']==0)]
    temp_dispatchregionsum.sort_values(by=['SETTLEMENTDATE'],ascending=True,inplace=True)
    return temp_dispatchregionsum.drop(columns=['INTERVENTION']).drop_duplicates()

In [9]:
def get_simulated_dispatch_price(reducedformsupply,settlementdate,residualdemand,PriceCap):
    supplycurve = reducedformsupply[reducedformsupply['SETTLEMENTDATE']==settlementdate]
    supplycurve['AccCapacity'] = supplycurve['AVAILABILITY'].cumsum()
    try:
        price = supplycurve[supplycurve['AccCapacity']>=residualdemand]['SRMC'].values[0]
    except:
        price = PriceCap
    return price

In [10]:
def read_dispatchprice(year,month,file_path,regionid):
    temp_dispatchprice = pd.read_csv(f'{file_path}Raw_data_zipped/DISPATCHPRICE/PUBLIC_DVD_DISPATCHPRICE_{str(year)}{str(month).zfill(2)}010000.zip',
                                     skiprows=1,
                                     parse_dates=['SETTLEMENTDATE'],
                                     usecols=['SETTLEMENTDATE','REGIONID','INTERVENTION','RRP'])
    temp_dispatchprice = temp_dispatchprice[(temp_dispatchprice['INTERVENTION']==0)&
                                            (temp_dispatchprice['REGIONID']==regionid)].drop(columns=['INTERVENTION'])
    return temp_dispatchprice.drop_duplicates()

# Generating the simulated RRP

This section:
- Simulate the dispatch price (RRP) for:
    - SRMC bids
    - Actual bids

## SRMC supply curve

In [11]:
# # Test purpose for generating average supply curve

# # Currently not in use
# demand_points_NSW1 = list(range(100,12500,300))
# demand_points_QLD1 = list(range(100,11500,300))
# demand_points_VIC1 = list(range(100,9100,200))
# demand_points_SA1 = list(range(100,3500,100))

In [11]:
def derive_SRMC_supply_curve_workflow(file_path,Year,Month,Regionid):
    
    """
    The adjustment made on the battery/hydro is equivalent to assuming them as price takers.
    Except in the few cases when the TOTALCLEARED > AVAILABILITY.
    """
    
    duid_mapping = get_regional_duid_mapping(file_path,Regionid)
    SRMC_regional = get_generator_srmc(Year,Month,file_path,Regionid)
    PriceCap = market_price_cap_mapping[YearMonthFY[(Year,Month)]]
    
    # Read and process DISPATCHLOAD
    temp_dispatch = read_dispatchload(Year,Month,file_path,duid_mapping)
    temp_dispatch = temp_dispatch.merge(right=duid_mapping[['DUID','REGIONID','DISPATCHTYPE']],
                                        on=['DUID'],
                                        how='left')\
                                 .merge(right=SRMC_regional[['DUID','FUELTYPE','SRMC']],
                                        on=['DUID'],
                                        how='left')
    temp_dispatch = temp_dispatch[(temp_dispatch['DISPATCHTYPE']=='GENERATOR')&
                                  (temp_dispatch['FUELTYPE']!=0)]

    # For battery and hydro, adjust their availability to their targets
    temp_dispatch['AVAILABILITY'] = temp_dispatch.parallel_apply(lambda row: row['TOTALCLEARED'] if row['FUELTYPE'] in ['Battery','Hydro'] else row['AVAILABILITY'],axis=1)
    # For battery and hydro, adjust their SRMC cost to 0
    temp_dispatch['SRMC'] = temp_dispatch.parallel_apply(lambda row: 0 if row['FUELTYPE'] in ['Battery','Hydro'] else row['SRMC'],axis=1)
    
    # Create reduced form supply
    Supply_SRMC = temp_dispatch.drop(columns=['TOTALCLEARED','REGIONID','DISPATCHTYPE'])\
                               .sort_values(by=['SETTLEMENTDATE','SRMC'],ascending=True)
    
    
    # Read and process DISPATCH REGIONSUM data
    temp_dispatchregionsum = read_dispatchregionsum(Year,Month,file_path,Regionid)
    temp_dispatchregionsum['RegionalTarget'] = temp_dispatchregionsum['TOTALDEMAND']+\
                                               temp_dispatchregionsum['NETINTERCHANGE']+\
                                               temp_dispatchregionsum['DISPATCHABLELOAD']
    temp_dispatchregionsum['Year'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).year)
    temp_dispatchregionsum['Month'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).month)
    temp_dispatchregionsum['FY'] = temp_dispatchregionsum.parallel_apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
    temp_dispatchregionsum['Quarter'] = temp_dispatchregionsum['Month'].parallel_apply(lambda x: 'Q'+str(math.ceil(x/3)))
    temp_dispatchregionsum['Hour'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).hour)
    
    # Find the simulated RRP for each DI
    temp_dispatchregionsum['RRP_simulated'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price(Supply_SRMC,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                              axis=1)
    
#     # Find the simulated RRP for each demand point
#     # This is for generating the derived demand curve, not in use for now.
#     for demand_point in demand_points_SA1:
#         temp_dispatchregionsum[demand_point] = temp_dispatchregionsum\
#                                                     .parallel_apply(lambda row: get_simulated_dispatch_price(Supply_SRMC,row['SETTLEMENTDATE'],demand_point,PriceCap),axis=1)    
    
    return temp_dispatchregionsum, Supply_SRMC

## ACTUAL supply curve

In [13]:
# def return_maxavail_adjusted_avail(row):
#     """
#     Function under TEST.
#     This function returns the availability for each price band 
#     after considering the effect of MAXAVAIL and FIXEDLOAD.
#     The MAXAVAIL has been replaced by AVAILABILITY from DISPATCHLOAD.
    
#     Update:
#     Also considers ROCUP and ROCDOWN
#     """
    
#     # Initiate a zero avail list
#     adjusted_avails = [0,0,0,0,0,
#                        0,0,0,0,0]
    
#     # Set the minload
#     minload = max(0,row['INITIALMW']-5*row['ROCDOWN'])
    
#     # Set the cap of avail
#     # If the duid has a FIXEDLOAD than its bid does not enter the price setting process
#     if row['FIXEDLOAD'] > 0:
#         total_avail = 0
#     else:
#         # The maxload is the min between MAXAVAIL (AVAILABILITY) and INITIALMW + 5*ROCUP
#         total_avail = min(row['MAXAVAIL'],row['INITIALMW']+5*row['ROCUP'])
    
#     accumulated_avail = 0

#     for i in range(1,11):
#         if row[f'BANDAVAIL{str(i)}'] <= 0:
#             adjusted_avails[i-1] = row[f'BANDAVAIL{str(i)}']
#         elif row[f'BANDAVAIL{str(i)}'] > 0:
#             row[f'BANDAVAIL{str(i)}'] = max(minload,row[f'BANDAVAIL{str(i)}'])
#             minload = 0
#             if accumulated_avail + row[f'BANDAVAIL{str(i)}'] < total_avail:
#                 accumulated_avail+=row[f'BANDAVAIL{str(i)}']
#                 adjusted_avails[i-1] = row[f'BANDAVAIL{str(i)}']
#             else:
#                 avail_space = total_avail - accumulated_avail
#                 accumulated_avail=total_avail
#                 adjusted_avails[i-1] = avail_space
#                 break
#     return adjusted_avails

In [11]:
def return_maxavail_adjusted_avail(row):
    """
    Function under TEST.
    This function returns the availability for each price band 
    after considering the effect of MAXAVAIL and FIXEDLOAD.
    The MAXAVAIL has been replaced by AVAILABILITY from DISPATCHLOAD.
    """
    
    # Initiate a zero avail list
    adjusted_avails = [0,0,0,0,0,
                       0,0,0,0,0]
    
    # Set the cap of avail
    # If the duid has a FIXEDLOAD than its bid does not enter the price setting process
    if row['FIXEDLOAD'] > 0:
        total_avail = 0
    else:
        total_avail = row['MAXAVAIL']
    
    accumulated_avail = 0
    
    for i in range(1,11):
        if accumulated_avail + row[f'BANDAVAIL{str(i)}'] < total_avail:
            accumulated_avail+=row[f'BANDAVAIL{str(i)}']
            adjusted_avails[i-1] = row[f'BANDAVAIL{str(i)}']
        else:
            avail_space = total_avail - accumulated_avail
            accumulated_avail=total_avail
            adjusted_avails[i-1] = avail_space
            break
    return adjusted_avails

In [12]:
def return_price_bands(row):
    """
    This function convert 10 price bands into a list for further calculation.
    """
    price_bands = [row[f'PRICEBAND{str(i)}'] for i in range(1,11)]
    return price_bands

def read_bid_dayoffer(bid_dayoffer_path,year,month,duid_mapping):
    # Read bid day offer
    try:
        temp_bid_dayoffer = pd.read_csv(f'{bid_dayoffer_path}/PUBLIC_DVD_BIDDAYOFFER_D_{str(year)}{str(month).zfill(2)}010000.zip',
                                        skiprows=0,
                                        usecols=['SETTLEMENTDATE','DUID','BIDTYPE',
                                                 'PRICEBAND1', 'PRICEBAND2','PRICEBAND3', 'PRICEBAND4', 'PRICEBAND5', 
                                                 'PRICEBAND6', 'PRICEBAND7','PRICEBAND8', 'PRICEBAND9', 'PRICEBAND10'],
                                        parse_dates=['SETTLEMENTDATE']).dropna()
    except:
        temp_bid_dayoffer = pd.read_csv(f'{bid_dayoffer_path}/PUBLIC_DVD_BIDDAYOFFER_D_{str(year)}{str(month).zfill(2)}010000.zip',
                                        skiprows=1,
                                        usecols=['SETTLEMENTDATE','DUID','BIDTYPE',
                                                 'PRICEBAND1', 'PRICEBAND2','PRICEBAND3', 'PRICEBAND4', 'PRICEBAND5', 
                                                 'PRICEBAND6', 'PRICEBAND7','PRICEBAND8', 'PRICEBAND9', 'PRICEBAND10'],
                                        parse_dates=['SETTLEMENTDATE']).dropna()
        
    temp_bid_dayoffer = temp_bid_dayoffer[(temp_bid_dayoffer['BIDTYPE']=='ENERGY')&
                                          (temp_bid_dayoffer['DUID'].isin(duid_mapping['DUID'].values))]\
                                         .drop_duplicates(keep='first')
    return temp_bid_dayoffer

def read_bid_peroffer(bid_peroffer_path,year,month,duid_mapping):
    # Read bid per offer
    try:
        temp_bid_peroffer = pd.read_csv(f'{bid_peroffer_path}/PUBLIC_DVD_BIDPEROFFER_D_{str(year)}{str(month).zfill(2)}010000.zip',
                                        skiprows=0,
                                        compression='zip',
                                        usecols=['SETTLEMENTDATE','INTERVAL_DATETIME','DUID','BIDTYPE','MAXAVAIL','FIXEDLOAD',
                                                 'ROCUP','ROCDOWN',
                                                 'BANDAVAIL1', 'BANDAVAIL2','BANDAVAIL3', 'BANDAVAIL4', 'BANDAVAIL5', 
                                                 'BANDAVAIL6', 'BANDAVAIL7','BANDAVAIL8', 'BANDAVAIL9', 'BANDAVAIL10'],
                                        parse_dates=['SETTLEMENTDATE','INTERVAL_DATETIME']).dropna()
    except:
        temp_bid_peroffer = pd.read_csv(f'{bid_peroffer_path}/PUBLIC_DVD_BIDPEROFFER_D_{str(year)}{str(month).zfill(2)}010000.zip',
                                        skiprows=1,
                                        compression='zip',
                                        usecols=['SETTLEMENTDATE','INTERVAL_DATETIME','DUID','BIDTYPE','MAXAVAIL','FIXEDLOAD',
                                                 'ROCUP','ROCDOWN',
                                                 'BANDAVAIL1', 'BANDAVAIL2','BANDAVAIL3', 'BANDAVAIL4', 'BANDAVAIL5', 
                                                 'BANDAVAIL6', 'BANDAVAIL7','BANDAVAIL8', 'BANDAVAIL9', 'BANDAVAIL10'],
                                        parse_dates=['SETTLEMENTDATE','INTERVAL_DATETIME']).dropna()
        
    temp_bid_peroffer = temp_bid_peroffer[(temp_bid_peroffer['BIDTYPE']=='ENERGY')&
                                          (temp_bid_peroffer['DUID'].isin(duid_mapping['DUID'].values))]\
                                        .drop_duplicates(keep='first')
    return temp_bid_peroffer


# Define a function to merge data and do calculations for monthly data
def merge_bid_price_and_avail(file_path, bid_dayoffer_path,bid_peroffer_path,year, month, duid_mapping):
    
    """
    Make sure to choose the proper 'return_maxavail_adjusted_avail' function.
    """
    
    # Read dispatchload data, use AVAILABILITY to replace MAXAVAIL in BIDPEROFFER
    temp_dispatchload = read_dispatchload(year,month,file_path,duid_mapping)\
                            .drop(columns=['TOTALCLEARED'])\
                            .rename(columns={'SETTLEMENTDATE':'INTERVAL_DATETIME'})
    if month == 1:
        year_prev = year-1
        month_prev = 12
    else:
        year_prev = year
        month_prev = month-1
    
    # Read bid day offer
    temp_bid_dayoffer = read_bid_dayoffer(bid_dayoffer_path,year,month,duid_mapping)
    temp_bid_dayoffer_prev = read_bid_dayoffer(bid_dayoffer_path,year_prev,month_prev,duid_mapping)
    temp_bid_dayoffer = pd.concat([temp_bid_dayoffer_prev,temp_bid_dayoffer])
    del temp_bid_dayoffer_prev
    
    # Read bid per offer
    temp_bid_peroffer = read_bid_peroffer(bid_peroffer_path,year,month,duid_mapping)
    temp_bid_peroffer_prev = read_bid_peroffer(bid_peroffer_path,year_prev,month_prev,duid_mapping)
    temp_bid_peroffer = pd.concat([temp_bid_peroffer_prev,temp_bid_peroffer])
    del temp_bid_peroffer_prev
    
    # Combine bid day offer and per offer
    temp_bid_combined = temp_bid_peroffer.merge(right=temp_bid_dayoffer,
                                                on=['SETTLEMENTDATE','BIDTYPE','DUID'],
                                                how='left')
    # Combine with DISPATCHLOAD
    temp_bid_combined = temp_bid_combined.merge(right=temp_dispatchload,
                                                on=['INTERVAL_DATETIME','DUID'],
                                                how='left')
    temp_bid_combined = temp_bid_combined.drop(columns=['MAXAVAIL']).rename(columns={'AVAILABILITY':'MAXAVAIL'})
    
    temp_bid_combined = temp_bid_combined.dropna(subset=['MAXAVAIL'])
    
    
    del temp_bid_dayoffer, temp_bid_peroffer, temp_dispatchload
    
    temp_bid_combined['Bid_Prices'] = temp_bid_combined.parallel_apply(lambda row: return_price_bands(row),axis=1)
    temp_bid_combined['Bid_Avails'] = temp_bid_combined.parallel_apply(lambda row: return_maxavail_adjusted_avail(row),axis=1) 
    
    return temp_bid_combined[['INTERVAL_DATETIME','DUID','FIXEDLOAD','MAXAVAIL',
                              'Bid_Prices','Bid_Avails']]

# Adjust bid prices using MLF
def read_MLF_data(file_path,year,month,regionid,YearMonthFY):
    """
    This function reads the MLF data.
    The MLF data will be used to adjust bid prices before they are used in constructing supply and demand curves.
    """
    FY = YearMonthFY[(year,month)]
    FY_start_year = FY[2:].split('_')[0]
    FY_end_year = FY[2:].split('_')[1]
    
    MLF_data_path = f'{file_path}MLF/MLF_FY_{FY_start_year}_{FY_end_year}_Formatted.xlsx'
    
    # Only read generator MLF
    MLF_df = pd.read_excel(MLF_data_path,
                           sheet_name='{} Gen'.format(regionid),
                           skiprows=0,
                           usecols=['DUID','{}-{} MLF'.format(str(FY_start_year),str(FY_end_year)[-2:])])\
              .rename(columns={'{}-{} MLF'.format(str(FY_start_year),str(FY_end_year)[-2:]):'MLF'})
    return MLF_df

In [13]:
def get_simulated_dispatch_price_actual(supply_curve,settlementdate,demand,PriceCap):
    supplycurve = supply_curve[supply_curve['SETTLEMENTDATE']==settlementdate]
    supplycurve['AccCapacity'] = supplycurve['Bid_Avails'].cumsum()
    try:
        price = supplycurve[supplycurve['AccCapacity']>=demand]['Bid_Prices'].values[0]
    except:
        price = PriceCap
    return price

In [15]:
def derive_simulated_price_workflow_actual_individual_no_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                 Year,Month,Regionid,level):
    """
    This version of workflow allows specifying different levels to test the P1_alt.
    Available levels: DUID, STATIONNAME, PARTICIPANTNAME
    """
    
    
    # Read duid SRMC
    srmc_mapping = get_generator_srmc(Year,Month,file_path,Regionid)
    # Only keep the DUIDs other than storage
    srmc_mapping = srmc_mapping[~srmc_mapping['FUELTYPE'].isin(['Battery','Hydro'])]
    # Convert to dict for faster query
    srmc_mapping_dict = srmc_mapping.set_index(['DUID'])['SRMC'].to_dict()
    
    # Read duid mapping
    duid_mapping = get_regional_duid_mapping(file_path,Regionid)
    duid_mapping = duid_mapping[duid_mapping['DISPATCHTYPE']=='GENERATOR'][['DUID','STATIONNAME','PARTICIPANTNAME',
                                                                            'FUELTYPE','REGIONID','DISPATCHTYPE']]
    
    srmc_mapping_combined = srmc_mapping.merge(right=duid_mapping.drop(columns=['FUELTYPE','REGIONID','DISPATCHTYPE']),
                                                  on=['DUID'],
                                                  how='left')
    
    duid_dict = srmc_mapping_combined.groupby(by=[level])['DUID'].unique().to_dict()
    
    
    # Read, combine and process bid data
    temp_bid = merge_bid_price_and_avail(file_path, bid_dayoffer_path,bid_peroffer_path,Year,Month,duid_mapping)
    
    # Get the DI level FIXEDLOAD
    temp_fixedload = temp_bid.groupby(by=['INTERVAL_DATETIME'],as_index=False)[['FIXEDLOAD']]\
                             .sum()\
                             .rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    # Get the corresponding price cap    
    PriceCap = market_price_cap_mapping[YearMonthFY[(Year,Month)]]
    
    # Merge the bid data with MLF
    temp_MLF = read_MLF_data(file_path,Year,Month,Regionid,YearMonthFY)

    temp_bid = temp_bid.merge(right=temp_MLF,
                     on=['DUID'],
                     how='left')
    temp_bid['MLF'] = temp_bid['MLF'].fillna(1)

    # Adjust the bid prices using MLF
    # Convert prices to a NumPy array
    prices_array = np.array(temp_bid['Bid_Prices'].tolist())
    # Reshape factor for broadcasting (if necessary)
    factors = temp_bid['MLF'].values[:, np.newaxis]
    # Broadcast multiplication
    adjusted_prices = np.around(prices_array / factors,2)
    # Update DataFrame
    temp_bid['Bid_Prices'] = adjusted_prices.tolist()
    # Drop the MLF column
    temp_bid = temp_bid.drop(columns=['MLF'])
    
    # Reformat to get the supply curve
    Supply_ACTUAL = temp_bid.drop(columns=['MAXAVAIL','FIXEDLOAD']).explode(['Bid_Prices','Bid_Avails'])
    Supply_ACTUAL = Supply_ACTUAL[Supply_ACTUAL['Bid_Avails']>0]
    Supply_ACTUAL = Supply_ACTUAL.sort_values(by=['INTERVAL_DATETIME','Bid_Prices'],ascending=True)
    # Adjust using price cap
    Supply_ACTUAL['Bid_Prices'] = Supply_ACTUAL['Bid_Prices'].parallel_apply(lambda x: PriceCap if x>PriceCap else x)
    Supply_ACTUAL = Supply_ACTUAL.rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    
    # Read and process DISPATCH REGIONSUM data
    temp_dispatchregionsum = read_dispatchregionsum(Year,Month,file_path,Regionid)
    
    
    
        # Merge with FIXEDLOAD
    temp_dispatchregionsum = temp_dispatchregionsum.merge(right=temp_fixedload,
                                                          on=['SETTLEMENTDATE'],
                                                          how='left')
    
    temp_dispatchregionsum['RegionalTarget'] = temp_dispatchregionsum['TOTALDEMAND']+\
                                               temp_dispatchregionsum['NETINTERCHANGE']+\
                                               temp_dispatchregionsum['DISPATCHABLELOAD']-\
                                               temp_dispatchregionsum['FIXEDLOAD']
    temp_dispatchregionsum['Year'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).year)
    temp_dispatchregionsum['Month'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).month)
    temp_dispatchregionsum['FY'] = temp_dispatchregionsum.parallel_apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
    temp_dispatchregionsum['Quarter'] = temp_dispatchregionsum['Month'].parallel_apply(lambda x: 'Q'+str(math.ceil(x/3)))
    temp_dispatchregionsum['Hour'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).hour)

    
    # Find the simulated RRP for each DI
    temp_dispatchregionsum['RRP_simulated'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price_actual(Supply_ACTUAL,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                              axis=1)
    
    for level_unit in duid_dict.keys():
#         print(level_unit)
        try:
            Supply_ACTUAL_alt = Supply_ACTUAL.copy()
            for duid in duid_dict[level_unit]:
#                 print(level_unit,duid)
                Supply_ACTUAL_alt['Bid_Prices'] = Supply_ACTUAL_alt.apply(lambda row:srmc_mapping_dict[duid] if row['DUID'] == duid else row['Bid_Prices'], axis=1)
            Supply_ACTUAL_alt = Supply_ACTUAL_alt.sort_values(by=['SETTLEMENTDATE','Bid_Prices'],ascending=True)
            temp_dispatchregionsum[f'RRP_simulated_{level_unit}'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price_actual(Supply_ACTUAL_alt,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                                                                      axis=1)
        except:
            print(level_unit, 'error')
    
    return temp_dispatchregionsum,Supply_ACTUAL

In [14]:
def derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                 Year,Month,Regionid,level,storage_type,window_days=0):
    """
    This version of workflow allows specifying different levels to test the P1_alt.
    Available levels: DUID, STATIONNAME, PARTICIPANTNAME
    
    Storage_type can be: [Battery, Hydro_gas, Hydro_pumped, Hydro_river]
    For Hydro_gas, window_days can be [0, 99] (No longer in use)
    For Hydro_gravity, window_days = 0
    For Hydro_river, window_days = 0
    For Battery, window_days can be [1,3,5]
    For Hydro_pumped, window_days can be [7, 14, 21, 28]
    """
    
    
    # Read duid mapping
    duid_mapping = get_regional_duid_mapping(file_path,Regionid)
    duid_mapping = duid_mapping[duid_mapping['DISPATCHTYPE']=='GENERATOR'][['DUID','STATIONNAME','PARTICIPANTNAME',
                                                                            'FUELTYPE','REGIONID','DISPATCHTYPE']]
    
    # Create a mapping dict to map STATIONNAME to a list of DUIDs (Contains all battery and hydro, need to further filter)
    duid_storage_dict = duid_mapping[duid_mapping['FUELTYPE'].isin(['Battery','Hydro'])]\
                        .groupby(by=[level])['DUID'].unique().to_dict()

    
    
    # Read, combine and process bid data
    temp_bid = merge_bid_price_and_avail(file_path, bid_dayoffer_path,bid_peroffer_path,Year,Month,duid_mapping)
    
    # Get the DI level FIXEDLOAD
    temp_fixedload = temp_bid.groupby(by=['INTERVAL_DATETIME'],as_index=False)[['FIXEDLOAD']]\
                             .sum()\
                             .rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    # Get the corresponding price cap    
    PriceCap = market_price_cap_mapping[YearMonthFY[(Year,Month)]]
    
    # Merge the bid data with MLF
    temp_MLF = read_MLF_data(file_path,Year,Month,Regionid,YearMonthFY)

    temp_bid = temp_bid.merge(right=temp_MLF,
                     on=['DUID'],
                     how='left')
    temp_bid['MLF'] = temp_bid['MLF'].fillna(1)

    # Adjust the bid prices using MLF
    # Convert prices to a NumPy array
    prices_array = np.array(temp_bid['Bid_Prices'].tolist())
    # Reshape factor for broadcasting (if necessary)
    factors = temp_bid['MLF'].values[:, np.newaxis]
    # Broadcast multiplication
    adjusted_prices = np.around(prices_array / factors,2)
    # Update DataFrame
    temp_bid['Bid_Prices'] = adjusted_prices.tolist()
    # Drop the MLF column
    temp_bid = temp_bid.drop(columns=['MLF'])
    
    # Reformat to get the supply curve
    Supply_ACTUAL = temp_bid.drop(columns=['MAXAVAIL','FIXEDLOAD']).explode(['Bid_Prices','Bid_Avails'])
    Supply_ACTUAL = Supply_ACTUAL[Supply_ACTUAL['Bid_Avails']>0]
    Supply_ACTUAL = Supply_ACTUAL.sort_values(by=['INTERVAL_DATETIME','Bid_Prices'],ascending=True)
    # Adjust using price cap
    Supply_ACTUAL['Bid_Prices'] = Supply_ACTUAL['Bid_Prices'].parallel_apply(lambda x: PriceCap if x>PriceCap else x)
    Supply_ACTUAL = Supply_ACTUAL.rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    # Generate DateTime, for mapping with storage SRMC
    Supply_ACTUAL['DateTime'] = Supply_ACTUAL['SETTLEMENTDATE'].apply(lambda x: (x-datetime.timedelta(minutes=5)).date())
    Supply_ACTUAL['DateTime'] = Supply_ACTUAL['DateTime'].apply(lambda x: datetime.datetime(x.year,x.month,x.day))
    
    # Read and process DISPATCH REGIONSUM data
    temp_dispatchregionsum = read_dispatchregionsum(Year,Month,file_path,Regionid)
    
    
    
        # Merge with FIXEDLOAD
    temp_dispatchregionsum = temp_dispatchregionsum.merge(right=temp_fixedload,
                                                          on=['SETTLEMENTDATE'],
                                                          how='left')
    
    temp_dispatchregionsum['RegionalTarget'] = temp_dispatchregionsum['TOTALDEMAND']+\
                                               temp_dispatchregionsum['NETINTERCHANGE']+\
                                               temp_dispatchregionsum['DISPATCHABLELOAD']-\
                                               temp_dispatchregionsum['FIXEDLOAD']
    temp_dispatchregionsum['Year'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).year)
    temp_dispatchregionsum['Month'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).month)
    temp_dispatchregionsum['FY'] = temp_dispatchregionsum.parallel_apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
    temp_dispatchregionsum['Quarter'] = temp_dispatchregionsum['Month'].parallel_apply(lambda x: 'Q'+str(math.ceil(x/3)))
    temp_dispatchregionsum['Hour'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).hour)

    
    # Find the simulated RRP for each DI
    temp_dispatchregionsum['RRP_simulated'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price_actual(Supply_ACTUAL,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                              axis=1)
    
    # Read SRMC for the corresponding storage type and window days
    storage_srmc = pd.read_csv(f'{file_path}/Lerner_Index/Estimated_Storage_SRMC/{storage_type}/{storage_type}_srmc_{Regionid}_{window_days}.csv')
#                                parse_dates=['DateTime'],dayfirst=True
    storage_srmc['DateTime'] = pd.to_datetime(storage_srmc['DateTime'])
    storage_duids = list(storage_srmc['DUID'].unique())
    storage_srmc_dict = storage_srmc.set_index(['DUID','DateTime'])['SRMC'].to_dict()
    print(storage_duids)
    
    for level_unit in duid_storage_dict.keys():
        duid_exist_count = 0
        try:
            Supply_ACTUAL_alt = Supply_ACTUAL.copy()
            for duid in duid_storage_dict[level_unit]:
                if duid in storage_duids:
                    duid_exist_count += 1
                    Supply_ACTUAL_alt['Bid_Prices'] = Supply_ACTUAL_alt.apply(lambda row:storage_srmc_dict[(row['DUID'],row['DateTime'])] if row['DUID']==duid else row['Bid_Prices'], axis=1)
            if duid_exist_count > 0:
                Supply_ACTUAL_alt = Supply_ACTUAL_alt.sort_values(by=['SETTLEMENTDATE','Bid_Prices'],ascending=True)
                temp_dispatchregionsum[f'RRP_simulated_{level_unit}'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price_actual(Supply_ACTUAL_alt,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                                                                      axis=1)
        except:
            print(level_unit, 'error')
    
    return temp_dispatchregionsum,Supply_ACTUAL

In [19]:
def derive_ACTUAL_supply_curve_workflow(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                        Year,Month,Regionid):
    """
    This function is for regional collective market power.
    """
    
    # Read duid mapping
    duid_mapping = get_regional_duid_mapping(file_path,Regionid)
    duid_mapping = duid_mapping[duid_mapping['DISPATCHTYPE']=='GENERATOR']
    # Read, combine and process bid data
    temp_bid = merge_bid_price_and_avail(file_path, bid_dayoffer_path,bid_peroffer_path,Year,Month,duid_mapping)
    
    # Get the DI level FIXEDLOAD
    temp_fixedload = temp_bid.groupby(by=['INTERVAL_DATETIME'],as_index=False)[['FIXEDLOAD']]\
                             .sum()\
                             .rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    # Get the corresponding price cap    
    PriceCap = market_price_cap_mapping[YearMonthFY[(Year,Month)]]
    
    # Merge the bid data with MLF
    temp_MLF = read_MLF_data(file_path,Year,Month,Regionid,YearMonthFY)

    temp_bid = temp_bid.merge(right=temp_MLF,
                     on=['DUID'],
                     how='left')
    temp_bid['MLF'] = temp_bid['MLF'].fillna(1)

    # Adjust the bid prices using MLF
    # Convert prices to a NumPy array
    prices_array = np.array(temp_bid['Bid_Prices'].tolist())
    # Reshape factor for broadcasting (if necessary)
    factors = temp_bid['MLF'].values[:, np.newaxis]
    # Broadcast multiplication
    adjusted_prices = np.around(prices_array / factors,2)
    # Update DataFrame
    temp_bid['Bid_Prices'] = adjusted_prices.tolist()
    # Drop the MLF column
    temp_bid = temp_bid.drop(columns=['MLF'])
    
    # Reformat to get the supply curve
    Supply_ACTUAL = temp_bid.drop(columns=['MAXAVAIL','FIXEDLOAD']).explode(['Bid_Prices','Bid_Avails'])
    Supply_ACTUAL = Supply_ACTUAL[Supply_ACTUAL['Bid_Avails']>0]
    Supply_ACTUAL = Supply_ACTUAL.sort_values(by=['INTERVAL_DATETIME','Bid_Prices'],ascending=True)
    # Adjust using price cap
    Supply_ACTUAL['Bid_Prices'] = Supply_ACTUAL['Bid_Prices'].parallel_apply(lambda x: PriceCap if x>PriceCap else x)
    Supply_ACTUAL = Supply_ACTUAL.rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    
    # Read and process DISPATCH REGIONSUM data
    temp_dispatchregionsum = read_dispatchregionsum(Year,Month,file_path,Regionid)
    
    
    
        # Merge with FIXEDLOAD
    temp_dispatchregionsum = temp_dispatchregionsum.merge(right=temp_fixedload,
                                                          on=['SETTLEMENTDATE'],
                                                          how='left')
    temp_dispatchregionsum_test = temp_dispatchregionsum.copy()
    
    temp_dispatchregionsum['RegionalTarget'] = temp_dispatchregionsum['TOTALDEMAND']+\
                                               temp_dispatchregionsum['NETINTERCHANGE']+\
                                               temp_dispatchregionsum['DISPATCHABLELOAD']-\
                                               temp_dispatchregionsum['FIXEDLOAD']
    temp_dispatchregionsum['Year'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).year)
    temp_dispatchregionsum['Month'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).month)
    temp_dispatchregionsum['FY'] = temp_dispatchregionsum.parallel_apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
    temp_dispatchregionsum['Quarter'] = temp_dispatchregionsum['Month'].parallel_apply(lambda x: 'Q'+str(math.ceil(x/3)))
    temp_dispatchregionsum['Hour'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).hour)

    
    # Find the simulated RRP for each DI
    temp_dispatchregionsum['RRP_simulated'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price_actual(Supply_ACTUAL,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                              axis=1)    

    return temp_dispatchregionsum, Supply_ACTUAL

In [15]:
def derive_simulated_price_workflow_actual_individual_storage_vwadp(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                    Year,Month,Regionid,level,storage_type,window_days=0):
    """
    This version of workflow allows specifying different levels to test the P1_alt.
    Available levels: DUID, STATIONNAME, PARTICIPANTNAME
    
    Storage_type can be: [Battery, Hydro_all]
    For Battery, window_days can be [1,3,5]
    For Hydro_pumped, window_days can be [7, 14, 21, 28]
    """
    
    
    # Read duid mapping
    duid_mapping = get_regional_duid_mapping(file_path,Regionid)
    duid_mapping = duid_mapping[duid_mapping['DISPATCHTYPE']=='GENERATOR'][['DUID','STATIONNAME','PARTICIPANTNAME',
                                                                            'FUELTYPE','REGIONID','DISPATCHTYPE']]
    
    # Create a mapping dict to map STATIONNAME to a list of DUIDs (Contains all battery and hydro, need to further filter)
    duid_storage_dict = duid_mapping[duid_mapping['FUELTYPE'].isin(['Battery','Hydro'])]\
                        .groupby(by=[level])['DUID'].unique().to_dict()

    
    
    # Read, combine and process bid data
    temp_bid = merge_bid_price_and_avail(file_path, bid_dayoffer_path,bid_peroffer_path,Year,Month,duid_mapping)
    
    # Get the DI level FIXEDLOAD
    temp_fixedload = temp_bid.groupby(by=['INTERVAL_DATETIME'],as_index=False)[['FIXEDLOAD']]\
                             .sum()\
                             .rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    # Get the corresponding price cap    
    PriceCap = market_price_cap_mapping[YearMonthFY[(Year,Month)]]
    
    # Merge the bid data with MLF
    temp_MLF = read_MLF_data(file_path,Year,Month,Regionid,YearMonthFY)

    temp_bid = temp_bid.merge(right=temp_MLF,
                     on=['DUID'],
                     how='left')
    temp_bid['MLF'] = temp_bid['MLF'].fillna(1)

    # Adjust the bid prices using MLF
    # Convert prices to a NumPy array
    prices_array = np.array(temp_bid['Bid_Prices'].tolist())
    # Reshape factor for broadcasting (if necessary)
    factors = temp_bid['MLF'].values[:, np.newaxis]
    # Broadcast multiplication
    adjusted_prices = np.around(prices_array / factors,2)
    # Update DataFrame
    temp_bid['Bid_Prices'] = adjusted_prices.tolist()
    # Drop the MLF column
    temp_bid = temp_bid.drop(columns=['MLF'])
    
    # Reformat to get the supply curve
    Supply_ACTUAL = temp_bid.drop(columns=['MAXAVAIL','FIXEDLOAD']).explode(['Bid_Prices','Bid_Avails'])
    Supply_ACTUAL = Supply_ACTUAL[Supply_ACTUAL['Bid_Avails']>0]
    Supply_ACTUAL = Supply_ACTUAL.sort_values(by=['INTERVAL_DATETIME','Bid_Prices'],ascending=True)
    # Adjust using price cap
    Supply_ACTUAL['Bid_Prices'] = Supply_ACTUAL['Bid_Prices'].parallel_apply(lambda x: PriceCap if x>PriceCap else x)
    Supply_ACTUAL = Supply_ACTUAL.rename(columns={'INTERVAL_DATETIME':'SETTLEMENTDATE'})
    
    # Generate DateTime, for mapping with storage SRMC
    Supply_ACTUAL['DateTime'] = Supply_ACTUAL['SETTLEMENTDATE'].apply(lambda x: (x-datetime.timedelta(minutes=5)).date())
    Supply_ACTUAL['DateTime'] = Supply_ACTUAL['DateTime'].apply(lambda x: datetime.datetime(x.year,x.month,x.day))
    
    # Read and process DISPATCH REGIONSUM data
    temp_dispatchregionsum = read_dispatchregionsum(Year,Month,file_path,Regionid)
    
    
    
        # Merge with FIXEDLOAD
    temp_dispatchregionsum = temp_dispatchregionsum.merge(right=temp_fixedload,
                                                          on=['SETTLEMENTDATE'],
                                                          how='left')
    
    temp_dispatchregionsum['RegionalTarget'] = temp_dispatchregionsum['TOTALDEMAND']+\
                                               temp_dispatchregionsum['NETINTERCHANGE']+\
                                               temp_dispatchregionsum['DISPATCHABLELOAD']-\
                                               temp_dispatchregionsum['FIXEDLOAD']
    temp_dispatchregionsum['Year'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).year)
    temp_dispatchregionsum['Month'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).month)
    temp_dispatchregionsum['FY'] = temp_dispatchregionsum.parallel_apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
    temp_dispatchregionsum['Quarter'] = temp_dispatchregionsum['Month'].parallel_apply(lambda x: 'Q'+str(math.ceil(x/3)))
    temp_dispatchregionsum['Hour'] = temp_dispatchregionsum['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).hour)

    
    # Find the simulated RRP for each DI
    temp_dispatchregionsum['RRP_simulated'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price_actual(Supply_ACTUAL,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                              axis=1)
    
    # Read SRMC for the corresponding storage type and window days
    storage_srmc = pd.read_csv(f'{file_path}/Lerner_Index/Estimated_Storage_SRMC/{storage_type}/{storage_type}_srmc_{Regionid}_{window_days}_VWADP_wins_98.csv')
#                                parse_dates=['DateTime'],dayfirst=True
    storage_srmc['DateTime'] = pd.to_datetime(storage_srmc['DateTime'])
    storage_duids = list(storage_srmc['DUID'].unique())
    storage_srmc_dict = storage_srmc.set_index(['DUID','DateTime'])['SRMC'].to_dict()
    print(storage_duids)
    
    for level_unit in duid_storage_dict.keys():
        duid_exist_count = 0
        try:
            Supply_ACTUAL_alt = Supply_ACTUAL.copy()
            for duid in duid_storage_dict[level_unit]:
                if duid in storage_duids:
                    duid_exist_count += 1
                    Supply_ACTUAL_alt['Bid_Prices'] = Supply_ACTUAL_alt.apply(lambda row:storage_srmc_dict[(row['DUID'],row['DateTime'])] if row['DUID']==duid else row['Bid_Prices'], axis=1)
            if duid_exist_count > 0:
                Supply_ACTUAL_alt = Supply_ACTUAL_alt.sort_values(by=['SETTLEMENTDATE','Bid_Prices'],ascending=True)
                temp_dispatchregionsum[f'RRP_simulated_{level_unit}'] = temp_dispatchregionsum.parallel_apply(lambda row: get_simulated_dispatch_price_actual(Supply_ACTUAL_alt,row['SETTLEMENTDATE'],row['RegionalTarget'],PriceCap),
                                                                                                      axis=1)
        except:
            print(level_unit, 'error')
    
    return temp_dispatchregionsum,Supply_ACTUAL

## Generate data

### Regional collective

#### SRMC bids

In [ ]:
for Regionid in ['SA1','NSW1','VIC1','QLD1']:
    print(f'Processing {Regionid}')
#     demand_points = locals()[f'demand_points_{Regionid}']
    
    supply_curve_SRMC = pd.DataFrame()
    result_regional = pd.DataFrame()
    for Year in range(2017,2024):
        if Year == 2017:
            for Month in range(7,13):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_SRMC, Supply_SRMC = derive_SRMC_supply_curve_workflow(file_path,Year,Month,Regionid)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_SRMC])
                supply_curve_SRMC = pd.concat([supply_curve_SRMC,Supply_SRMC])
        elif Year == 2023:
            for Month in range(1,7):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_SRMC, Supply_SRMC = derive_SRMC_supply_curve_workflow(file_path,Year,Month,Regionid)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_SRMC])
                supply_curve_SRMC = pd.concat([supply_curve_SRMC,Supply_SRMC])
        else:
            for Month in range(1,13):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_SRMC, Supply_SRMC = derive_SRMC_supply_curve_workflow(file_path,Year,Month,Regionid)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_SRMC])
                supply_curve_SRMC = pd.concat([supply_curve_SRMC,Supply_SRMC])

    result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_SRMC_Supply_Curve_{Regionid}.csv',index=False)
    supply_curve_SRMC.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_SRMC_Supply_Curve_{Regionid}_breakdown.csv',index=False)
print('Done')

In [ ]:
del result_regional, supply_curve_SRMC

#### Actual bids

In [ ]:
for Regionid in ['SA1','NSW1','VIC1','QLD1']:
    print(f'Processing {Regionid}')
#     demand_points = locals()[f'demand_points_{Regionid}']

    supply_curve_Actual = pd.DataFrame()
    result_regional = pd.DataFrame()
    for Year in range(2017,2024):
        if Year == 2017:
            for Month in range(7,13):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_actual,Supply_Actual = derive_ACTUAL_supply_curve_workflow(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                                    Year,Month,Regionid)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
        elif Year == 2023:
            for Month in range(1,7):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_actual,Supply_Actual = derive_ACTUAL_supply_curve_workflow(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                                    Year,Month,Regionid)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
        else:
            for Month in range(1,13):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_actual,Supply_Actual = derive_ACTUAL_supply_curve_workflow(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                                    Year,Month,Regionid)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

    result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}.csv',index=False)
    supply_curve_Actual.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_breakdown.csv',index=False)
print('Done')

### Regional individual

#### Actual bids - Individual DUID

Only generated DUID level for SA and NSW, as I think the station level data is a better level for analysis.

In [ ]:
# for Regionid in ['SA1','NSW1']:
#     print(f'Processing {Regionid}')
# #     demand_points = locals()[f'demand_points_{Regionid}']

#     supply_curve_Actual = pd.DataFrame()
#     result_regional = pd.DataFrame()
#     for Year in range(2017,2024):
#         if Year == 2017:
#             for Month in range(7,13):
#                 print(f'Processing {Year} {Month}')
#                 temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
#                                                                                     Year,Month,Regionid)
#                 result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
#                 supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
#         elif Year == 2023:
#             for Month in range(1,7):
#                 print(f'Processing {Year} {Month}')
#                 temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
#                                                                                     Year,Month,Regionid)
#                 result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
#                 supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
#         else:
#             for Month in range(1,13):
#                 print(f'Processing {Year} {Month}')
#                 temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
#                                                                                     Year,Month,Regionid)
#                 result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
#                 supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

#     result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_individual_test.csv',index=False)
# #     supply_curve_Actual.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_breakdown.csv',index=False)
# print('Done')

#### Actual bids - individual participant

Only tested on SA data, as the participant level is too broad. The combination of renewables and fossil fuels would deliver masked signal.

In [ ]:
# level = 'PARTICIPANTNAME'

# for Regionid in ['SA1']:
#     print(f'Processing {Regionid}')
# #     demand_points = locals()[f'demand_points_{Regionid}']

#     supply_curve_Actual = pd.DataFrame()
#     result_regional = pd.DataFrame()
#     for Year in range(2017,2024):
#         if Year == 2017:
#             for Month in range(7,13):
#                 print(f'Processing {Year} {Month}')
#                 temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
#                                                                         Year,Month,Regionid,level)
#                 result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
#                 supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
#         elif Year == 2023:
#             for Month in range(1,7):
#                 print(f'Processing {Year} {Month}')
#                 temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
#                                                                         Year,Month,Regionid,level)
#                 result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
#                 supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
#         else:
#             for Month in range(1,13):
#                 print(f'Processing {Year} {Month}')
#                 temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
#                                                                         Year,Month,Regionid,level)
#                 result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
#                 supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

#     result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}.csv',index=False)
# #     supply_curve_Actual.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_breakdown.csv',index=False)
# print('Done')

#### Actual Individual station

Not storage

In [ ]:
level = 'STATIONNAME'

for Regionid in ['SA1','NSW1']:
    print(f'Processing {Regionid}')
#     demand_points = locals()[f'demand_points_{Regionid}']

    supply_curve_Actual = pd.DataFrame()
    result_regional = pd.DataFrame()
    for Year in range(2017,2024):
        if Year == 2017:
            for Month in range(7,13):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_no_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                        Year,Month,Regionid,level)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
        elif Year == 2023:
            for Month in range(1,7):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_no_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                        Year,Month,Regionid,level)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
        else:
            for Month in range(1,13):
                print(f'Processing {Year} {Month}')
                temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_no_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                        Year,Month,Regionid,level)
                result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

    result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_alt.csv',index=False)
#     supply_curve_Actual.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_breakdown.csv',index=False)
print('Done')

Storage (Tangible SRMC)

In [20]:
storage_type = 'Hydro_river'

level = 'STATIONNAME'
window_days = [0]

for Regionid in ['QLD1','SA1','NSW1','VIC1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt.csv',index=False)


Processing QLD1
Processing 2017 7
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2017 8
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2017 9
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2017 10
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2017 11
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2017 12
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2018 1
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2018 2
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2018 3
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2018 4
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAREEYA3', 'KAREEYA4']
Processing 2018 5
['BARRON-1', 'BARRON-2', 'KAREEYA1', 'KAREEYA2', 'KAR

[]
Processing 2017 10
[]
Processing 2017 11
[]
Processing 2017 12
[]
Processing 2018 1
[]
Processing 2018 2
[]
Processing 2018 3
[]
Processing 2018 4
[]
Processing 2018 5
[]
Processing 2018 6
[]
Processing 2018 7
[]
Processing 2018 8
[]
Processing 2018 9
[]
Processing 2018 10
[]
Processing 2018 11
[]
Processing 2018 12
[]
Processing 2019 1
[]
Processing 2019 2
[]
Processing 2019 3
[]
Processing 2019 4
[]
Processing 2019 5
[]
Processing 2019 6
[]
Processing 2019 7
[]
Processing 2019 8
[]
Processing 2019 9
[]
Processing 2019 10
[]
Processing 2019 11
[]
Processing 2019 12
[]
Processing 2020 1
[]
Processing 2020 2
[]
Processing 2020 3
[]
Processing 2020 4
[]
Processing 2020 5
[]
Processing 2020 6
[]
Processing 2020 7
[]
Processing 2020 8
[]
Processing 2020 9
[]
Processing 2020 10
[]
Processing 2020 11
[]
Processing 2020 12
[]
Processing 2021 1
[]
Processing 2021 2
[]
Processing 2021 3
[]
Processing 2021 4
[]
Processing 2021 5
[]
Processing 2021 6
[]
Processing 2021 7
[]
Processing 2021 8
[

In [15]:
storage_type = 'Hydro_gravity'

level = 'STATIONNAME'
window_days = [0]

for Regionid in ['QLD1','SA1','NSW1','VIC1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt.csv',index=False)


Processing QLD1
Processing 2017 7
[]
Processing 2017 8
[]
Processing 2017 9
[]
Processing 2017 10
[]
Processing 2017 11
[]
Processing 2017 12
[]
Processing 2018 1
[]
Processing 2018 2
[]
Processing 2018 3
[]
Processing 2018 4
[]
Processing 2018 5
[]
Processing 2018 6
[]
Processing 2018 7
[]
Processing 2018 8
[]
Processing 2018 9
[]
Processing 2018 10
[]
Processing 2018 11
[]
Processing 2018 12
[]
Processing 2019 1
[]
Processing 2019 2
[]
Processing 2019 3
[]
Processing 2019 4
[]
Processing 2019 5
[]
Processing 2019 6
[]
Processing 2019 7
[]
Processing 2019 8
[]
Processing 2019 9
[]
Processing 2019 10
[]
Processing 2019 11
[]
Processing 2019 12
[]
Processing 2020 1
[]
Processing 2020 2
[]
Processing 2020 3
[]
Processing 2020 4
[]
Processing 2020 5
[]
Processing 2020 6
[]
Processing 2020 7
[]
Processing 2020 8
[]
Processing 2020 9
[]
Processing 2020 10
[]
Processing 2020 11
[]
Processing 2020 12
[]
Processing 2021 1
[]
Processing 2021 2
[]
Processing 2021 3
[]
Processing 2021 4
[]
Proces

In [21]:
storage_type = 'Hydro_gas'

level = 'STATIONNAME'
window_days = [0]

for Regionid in ['NSW1','VIC1','QLD1','SA1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt.csv',index=False)

Processing NSW1
Processing 2017 7
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2017 8
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2017 9
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2017 10
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2017 11
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2017 12
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 1
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 2
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 3
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 4
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 5
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 6
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 7
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 8
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Processing 2018 9
['BLOWERNG', 'GUTHEGA', 'HUMENSW', 'UPPTUMUT']
Proces

Processing 2019 9
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1', 'EILDON2', 'HUMEV', 'MCKAY1', 'MCKAY2', 'MURRAY', 'WKIEWA1', 'WKIEWA2']
Processing 2019 10
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1', 'EILDON2', 'HUMEV', 'MCKAY1', 'MCKAY2', 'MURRAY', 'WKIEWA1', 'WKIEWA2']
Processing 2019 11
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1', 'EILDON2', 'HUMEV', 'MCKAY1', 'MCKAY2', 'MURRAY', 'WKIEWA1', 'WKIEWA2']
Processing 2019 12
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1', 'EILDON2', 'HUMEV', 'MCKAY1', 'MCKAY2', 'MURRAY', 'WKIEWA1', 'WKIEWA2']
Processing 2020 1
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1', 'EILDON2', 'HUMEV', 'MCKAY1', 'MCKAY2', 'MURRAY', 'WKIEWA1', 'WKIEWA2']
Processing 2020 2
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1', 'EILDON2', 'HUMEV', 'MCKAY1', 'MCKAY2', 'MURRAY', 'WKIEWA1', 'WKIEWA2']
Processing 2020 3
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1', 'EILDON2', 'HUMEV', 'MCKAY1', 'MCKAY2', 'MURRAY', 'WKIEWA1', 'WKIEWA2']
Processing 2020 4
['CLOVER1', 'CLOVER2', 'DARTM1', 'EILDON1

[]
Processing 2019 5
[]
Processing 2019 6
[]
Processing 2019 7
[]
Processing 2019 8
[]
Processing 2019 9
[]
Processing 2019 10
[]
Processing 2019 11
[]
Processing 2019 12
[]
Processing 2020 1
[]
Processing 2020 2
[]
Processing 2020 3
[]
Processing 2020 4
[]
Processing 2020 5
[]
Processing 2020 6
[]
Processing 2020 7
[]
Processing 2020 8
[]
Processing 2020 9
[]
Processing 2020 10
[]
Processing 2020 11
[]
Processing 2020 12
[]
Processing 2021 1
[]
Processing 2021 2
[]
Processing 2021 3
[]
Processing 2021 4
[]
Processing 2021 5
[]
Processing 2021 6
[]
Processing 2021 7
[]
Processing 2021 8
[]
Processing 2021 9
[]
Processing 2021 10
[]
Processing 2021 11
[]
Processing 2021 12
[]
Processing 2022 1
[]
Processing 2022 2
[]
Processing 2022 3
[]
Processing 2022 4
[]
Processing 2022 5
[]
Processing 2022 6
[]
Processing 2022 7
[]
Processing 2022 8
[]
Processing 2022 9
[]
Processing 2022 10
[]
Processing 2022 11
[]
Processing 2022 12
[]
Processing 2023 1
[]
Processing 2023 2
[]
Processing 2023 3
[

In [17]:
storage_type = 'Hydro_gas'

level = 'STATIONNAME'
window_days = [99]

for Regionid in ['NSW1','VIC1','QLD1','SA1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt.csv',index=False)

Processing NSW1
Processing 2017 7
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2017 8
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2017 9
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2017 10
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2017 11
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2017 12
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 1
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 2
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 3
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 4
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 5
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 6
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 7
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 8
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 9
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 10
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 11
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Processing 2018 12
['BLOWERNG', 'GUTHEGA', 'UPPTUMUT']
Proces

In [22]:
storage_type = 'Hydro_pumped'

level = 'STATIONNAME'
window_days = [7,14,21,28]

for Regionid in ['NSW1','VIC1','QLD1','SA1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt.csv',index=False)

Processing NSW1
Processing 2017 7
['SHGEN', 'TUMUT3']
Processing 2017 8
['SHGEN', 'TUMUT3']
Processing 2017 9
['SHGEN', 'TUMUT3']
Processing 2017 10
['SHGEN', 'TUMUT3']
Processing 2017 11
['SHGEN', 'TUMUT3']
Processing 2017 12
['SHGEN', 'TUMUT3']
Processing 2018 1
['SHGEN', 'TUMUT3']
Processing 2018 2
['SHGEN', 'TUMUT3']
Processing 2018 3
['SHGEN', 'TUMUT3']
Processing 2018 4
['SHGEN', 'TUMUT3']
Processing 2018 5
['SHGEN', 'TUMUT3']
Processing 2018 6
['SHGEN', 'TUMUT3']
Processing 2018 7
['SHGEN', 'TUMUT3']
Processing 2018 8
['SHGEN', 'TUMUT3']
Processing 2018 9
['SHGEN', 'TUMUT3']
Processing 2018 10
['SHGEN', 'TUMUT3']
Processing 2018 11
['SHGEN', 'TUMUT3']
Processing 2018 12
['SHGEN', 'TUMUT3']
Processing 2019 1
['SHGEN', 'TUMUT3']
Processing 2019 2
['SHGEN', 'TUMUT3']
Processing 2019 3
['SHGEN', 'TUMUT3']
Processing 2019 4
['SHGEN', 'TUMUT3']
Processing 2019 5
['SHGEN', 'TUMUT3']
Processing 2019 6
['SHGEN', 'TUMUT3']
Processing 2019 7
['SHGEN', 'TUMUT3']
Processing 2019 8
['SHGEN', 

Processing 2023 4
['SHGEN', 'TUMUT3']
Processing 2023 5
['SHGEN', 'TUMUT3']
Processing 2023 6
['SHGEN', 'TUMUT3']
Processing NSW1
Processing 2017 7
['SHGEN', 'TUMUT3']
Processing 2017 8
['SHGEN', 'TUMUT3']
Processing 2017 9
['SHGEN', 'TUMUT3']
Processing 2017 10
['SHGEN', 'TUMUT3']
Processing 2017 11
['SHGEN', 'TUMUT3']
Processing 2017 12
['SHGEN', 'TUMUT3']
Processing 2018 1
['SHGEN', 'TUMUT3']
Processing 2018 2
['SHGEN', 'TUMUT3']
Processing 2018 3
['SHGEN', 'TUMUT3']
Processing 2018 4
['SHGEN', 'TUMUT3']
Processing 2018 5
['SHGEN', 'TUMUT3']
Processing 2018 6
['SHGEN', 'TUMUT3']
Processing 2018 7
['SHGEN', 'TUMUT3']
Processing 2018 8
['SHGEN', 'TUMUT3']
Processing 2018 9
['SHGEN', 'TUMUT3']
Processing 2018 10
['SHGEN', 'TUMUT3']
Processing 2018 11
['SHGEN', 'TUMUT3']
Processing 2018 12
['SHGEN', 'TUMUT3']
Processing 2019 1
['SHGEN', 'TUMUT3']
Processing 2019 2
['SHGEN', 'TUMUT3']
Processing 2019 3
['SHGEN', 'TUMUT3']
Processing 2019 4
['SHGEN', 'TUMUT3']
Processing 2019 5
['SHGEN', 

[]
Processing 2020 2
[]
Processing 2020 3
[]
Processing 2020 4
[]
Processing 2020 5
[]
Processing 2020 6
[]
Processing 2020 7
[]
Processing 2020 8
[]
Processing 2020 9
[]
Processing 2020 10
[]
Processing 2020 11
[]
Processing 2020 12
[]
Processing 2021 1
[]
Processing 2021 2
[]
Processing 2021 3
[]
Processing 2021 4
[]
Processing 2021 5
[]
Processing 2021 6
[]
Processing 2021 7
[]
Processing 2021 8
[]
Processing 2021 9
[]
Processing 2021 10
[]
Processing 2021 11
[]
Processing 2021 12
[]
Processing 2022 1
[]
Processing 2022 2
[]
Processing 2022 3
[]
Processing 2022 4
[]
Processing 2022 5
[]
Processing 2022 6
[]
Processing 2022 7
[]
Processing 2022 8
[]
Processing 2022 9
[]
Processing 2022 10
[]
Processing 2022 11
[]
Processing 2022 12
[]
Processing 2023 1
[]
Processing 2023 2
[]
Processing 2023 3
[]
Processing 2023 4
[]
Processing 2023 5
[]
Processing 2023 6
[]
Processing QLD1
Processing 2017 7
['W/HOE#1', 'W/HOE#2']
Processing 2017 8
['W/HOE#1', 'W/HOE#2']
Processing 2017 9
['W/HOE#1',

['W/HOE#1', 'W/HOE#2']
Processing 2020 4
['W/HOE#1', 'W/HOE#2']
Processing 2020 5
['W/HOE#1', 'W/HOE#2']
Processing 2020 6
['W/HOE#1', 'W/HOE#2']
Processing 2020 7
['W/HOE#1', 'W/HOE#2']
Processing 2020 8
['W/HOE#1', 'W/HOE#2']
Processing 2020 9
['W/HOE#1', 'W/HOE#2']
Processing 2020 10
['W/HOE#1', 'W/HOE#2']
Processing 2020 11
['W/HOE#1', 'W/HOE#2']
Processing 2020 12
['W/HOE#1', 'W/HOE#2']
Processing 2021 1
['W/HOE#1', 'W/HOE#2']
Processing 2021 2
['W/HOE#1', 'W/HOE#2']
Processing 2021 3
['W/HOE#1', 'W/HOE#2']
Processing 2021 4
['W/HOE#1', 'W/HOE#2']
Processing 2021 5
['W/HOE#1', 'W/HOE#2']
Processing 2021 6
['W/HOE#1', 'W/HOE#2']
Processing 2021 7
['W/HOE#1', 'W/HOE#2']
Processing 2021 8
['W/HOE#1', 'W/HOE#2']
Processing 2021 9
['W/HOE#1', 'W/HOE#2']
Processing 2021 10
['W/HOE#1', 'W/HOE#2']
Processing 2021 11
['W/HOE#1', 'W/HOE#2']
Processing 2021 12
['W/HOE#1', 'W/HOE#2']
Processing 2022 1
['W/HOE#1', 'W/HOE#2']
Processing 2022 2
['W/HOE#1', 'W/HOE#2']
Processing 2022 3
['W/HOE#1'

[]
Processing 2019 6
[]
Processing 2019 7
[]
Processing 2019 8
[]
Processing 2019 9
[]
Processing 2019 10
[]
Processing 2019 11
[]
Processing 2019 12
[]
Processing 2020 1
[]
Processing 2020 2
[]
Processing 2020 3
[]
Processing 2020 4
[]
Processing 2020 5
[]
Processing 2020 6
[]
Processing 2020 7
[]
Processing 2020 8
[]
Processing 2020 9
[]
Processing 2020 10
[]
Processing 2020 11
[]
Processing 2020 12
[]
Processing 2021 1
[]
Processing 2021 2
[]
Processing 2021 3
[]
Processing 2021 4
[]
Processing 2021 5
[]
Processing 2021 6
[]
Processing 2021 7
[]
Processing 2021 8
[]
Processing 2021 9
[]
Processing 2021 10
[]
Processing 2021 11
[]
Processing 2021 12
[]
Processing 2022 1
[]
Processing 2022 2
[]
Processing 2022 3
[]
Processing 2022 4
[]
Processing 2022 5
[]
Processing 2022 6
[]
Processing 2022 7
[]
Processing 2022 8
[]
Processing 2022 9
[]
Processing 2022 10
[]
Processing 2022 11
[]
Processing 2022 12
[]
Processing 2023 1
[]
Processing 2023 2
[]
Processing 2023 3
[]
Processing 2023 4
[

In [34]:
storage_type = 'Battery'

level = 'STATIONNAME'
window_days = [1,3,5]

for Regionid in ['SA1','NSW1','VIC1','QLD1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt.csv',index=False)

Processing SA1
Processing 2017 7
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 8
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 9
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 10
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 11
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 12
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 1
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 2
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 3
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 4
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', '

['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 3
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 4
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 5
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 6
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 7
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 8
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 9
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 10
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 11
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2B

Processing 2018 10
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 11
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 12
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2019 1
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2019 2
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2019 3
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2019 4
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2019 5
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2019 6
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2019 7
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPR

Processing 2020 2
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 3
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 4
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 5
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 6
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 7
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 8
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 9
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 10
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 11
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2020 12
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2021 1
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2021 2
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2021 3
['DPNTBG1', 'QBYN

['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2023 5
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2023 6
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing NSW1
Processing 2017 7
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2017 8
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2017 9
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2017 10
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2017 11
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2017 12
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2018 1
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2018 2
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2018 3
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2018 4
['DPNTBG1', 'QBYNBG1', 'RESS1G', 'RIVNBG2', 'WALGRVG1']
Processing 2018 5
['DPNTBG1', 'QBYNBG

Processing 2020 2
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 3
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 4
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 5
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 6
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 7
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 8
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 9
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 10
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 11
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2020 12
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2021 1
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1'

['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 4
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 5
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 6
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 7
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 8
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 9
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 10
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 11
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2022 12
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2023 1
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Processing 2023 2
['BALBG1', 'BULBESG1', 'GANNBG1', 'HBESSG1', 'PIBESSG1', 'VBBG1']
Process

['WANDBG1']
Processing 2019 9
['WANDBG1']
Processing 2019 10
['WANDBG1']
Processing 2019 11
['WANDBG1']
Processing 2019 12
['WANDBG1']
Processing 2020 1
['WANDBG1']
Processing 2020 2
['WANDBG1']
Processing 2020 3
['WANDBG1']
Processing 2020 4
['WANDBG1']
Processing 2020 5
['WANDBG1']
Processing 2020 6
['WANDBG1']
Processing 2020 7
['WANDBG1']
Processing 2020 8
['WANDBG1']
Processing 2020 9
['WANDBG1']
Processing 2020 10
['WANDBG1']
Processing 2020 11
['WANDBG1']
Processing 2020 12
['WANDBG1']
Processing 2021 1
['WANDBG1']
Processing 2021 2
['WANDBG1']
Processing 2021 3
['WANDBG1']
Processing 2021 4
['WANDBG1']
Processing 2021 5
['WANDBG1']
Processing 2021 6
['WANDBG1']
Processing 2021 7
['WANDBG1']
Processing 2021 8
['WANDBG1']
Processing 2021 9
['WANDBG1']
Processing 2021 10
['WANDBG1']
Processing 2021 11
['WANDBG1']
Processing 2021 12
['WANDBG1']
Processing 2022 1
['WANDBG1']
Processing 2022 2
['WANDBG1']
Processing 2022 3
['WANDBG1']
Processing 2022 4
['WANDBG1']
Processing 2022 5
[

Storage (Perceived SRMC)

Battery

In [17]:
storage_type = 'Battery'

level = 'STATIONNAME'
window_days = [1,3,5]

for Regionid in ['SA1','NSW1','VIC1','QLD1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage_vwadp(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage_vwadp(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage_vwadp(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt_vwadp.csv',index=False)

Processing SA1
Processing 2017 7
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 8
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 9
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 10
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 11
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2017 12
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 1
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 2
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH01', 'HPRG1', 'HVWWBA1G', 'LBBG1', 'TB2BG1', 'TIBG1']
Processing 2018 3
['ADPBA1G', 'BOWWBA1G', 'CBWWBA1G', 'DALNTH0

Hydro_all

In [18]:
storage_type = 'Hydro_all'

level = 'STATIONNAME'
window_days = [7,14,21,28]

for Regionid in ['SA1','NSW1','VIC1','QLD1']:
    for window_day in window_days:
        print(f'Processing {Regionid}')
    #     demand_points = locals()[f'demand_points_{Regionid}']

        supply_curve_Actual = pd.DataFrame()
        result_regional = pd.DataFrame()
        for Year in range(2017,2024):
            if Year == 2017:
                for Month in range(7,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage_vwadp(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            elif Year == 2023:
                for Month in range(1,7):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage_vwadp(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])
            else:
                for Month in range(1,13):
                    print(f'Processing {Year} {Month}')
                    temp_dispatchregionsum_actual,Supply_Actual = derive_simulated_price_workflow_actual_individual_storage_vwadp(file_path,bid_dayoffer_path,bid_peroffer_path,YearMonthFY,
                                                                            Year,Month,Regionid,level,storage_type,window_day)
                    result_regional = pd.concat([result_regional,temp_dispatchregionsum_actual])
                    supply_curve_Actual = pd.concat([supply_curve_Actual,Supply_Actual])

        result_regional.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_ACTUAL_Supply_Curve_{Regionid}_{level}_{storage_type}_{window_day}_alt_vwadp.csv',index=False)

Processing SA1
Processing 2017 7
[]
Processing 2017 8
[]
Processing 2017 9
[]
Processing 2017 10
[]
Processing 2017 11
[]
Processing 2017 12
[]
Processing 2018 1
[]
Processing 2018 2
[]
Processing 2018 3
[]
Processing 2018 4
[]
Processing 2018 5
[]
Processing 2018 6
[]
Processing 2018 7
[]
Processing 2018 8
[]
Processing 2018 9
[]
Processing 2018 10
[]
Processing 2018 11
[]
Processing 2018 12
[]
Processing 2019 1
[]
Processing 2019 2
[]
Processing 2019 3
[]
Processing 2019 4
[]
Processing 2019 5
[]
Processing 2019 6
[]
Processing 2019 7
[]
Processing 2019 8
[]
Processing 2019 9
[]
Processing 2019 10
[]
Processing 2019 11
[]
Processing 2019 12
[]
Processing 2020 1
[]
Processing 2020 2
[]
Processing 2020 3
[]
Processing 2020 4
[]
Processing 2020 5
[]
Processing 2020 6
[]
Processing 2020 7
[]
Processing 2020 8
[]
Processing 2020 9
[]
Processing 2020 10
[]
Processing 2020 11
[]
Processing 2020 12
[]
Processing 2021 1
[]
Processing 2021 2
[]
Processing 2021 3
[]
Processing 2021 4
[]
Process